## TrainingRunnner

- setup dataloader
- start training

## Import and Setup

In [27]:
from IPython.core.debugger import set_trace

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#export 
from exp.nb_CustomDataLoader import Detection_dataset

In [3]:
! pip install pycocotools

  Using cached pycocotools-2.0.1.tar.gz (23 kB)
  Running setup.py clean for pycocotools
Failed to build pycocotools
    Running setup.py install for pycocotools: started
    Running setup.py install for pycocotools: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Deployment\.conda\envs\yolo4\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Deployment\\AppData\\Local\\Temp\\pip-install-ucocx98p\\pycocotools\\setup.py'"'"'; __file__='"'"'C:\\Users\\Deployment\\AppData\\Local\\Temp\\pip-install-ucocx98p\\pycocotools\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Deployment\AppData\Local\Temp\pip-wheel-3lekh7vq'
       cwd: C:\Users\Deployment\AppData\Local\Temp\pip-install-ucocx98p\pycocotools\
  Complete output (22 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\pycocotools
  copying pycocotools\coco.py -> build\lib.win-amd64-3.8\pycocotools
  copying pycocotools\cocoeval.py -> build\lib.wi

In [4]:
# !pip install tensorboardX

In [5]:
import os, sys
sys.path.append("D:/YoloV5_Hui/poladrone_nb/ref/pytorchYOLOv4/")

In [6]:
# import torch
# import torch.nn as nn
# from torch.utils.data import DataLoader
# from torch import optim
# from tensorboardX import SummaryWriter
# import logging
# import os, sys
# from tqdm import tqdm
# # from dataset import Yolo_dataset
# # from cfg import Cfg__init
# from ref.pytorchYOLOv4.models import Yolov4

# import argparse
# from easydict import EasyDict as edict
# from torch.nn import functional as F

# import numpy as np

In [7]:
from ref.pytorchYOLOv4.train import *
import torch.nn as nn

## Function

In [28]:
def custom_collate(batch):
    images = []
    bboxes = []
    for i, (img, box) in enumerate(batch):
        images.append([img])
        
        box = np.concatenate((box, [[i]]*len(box)), 1)
        bboxes.append(box)
        
    images = np.concatenate(images, axis=0)
    images = images.transpose(0, 3, 1, 2)
    images = torch.from_numpy(images).div(255.0)
            
    bboxes = np.concatenate(bboxes)

    bboxes = torch.from_numpy(bboxes)
    return images, bboxes

In [29]:
def custom_train(train_dataset, val_dataset, model, device, config, epochs=5, batch_size=1, save_cp=True, log_step=20, img_scale=0.5):
#     train_dataset = Yolo_dataset(config.train_label, config)
#     val_dataset = Yolo_dataset(config.val_label, config)

    n_train = len(train_dataset)
    n_val = len(val_dataset)

    train_loader = DataLoader(train_dataset, batch_size=config.batch // config.subdivisions, shuffle=True,
                              num_workers=0, pin_memory=True, drop_last=True, collate_fn=custom_collate)

#     val_loader = DataLoader(val_dataset, batch_size=config.batch // config.subdivisions, shuffle=True, num_workers=1,
#                             pin_memory=True, drop_last=True)

    writer = SummaryWriter(log_dir=config.TRAIN_TENSORBOARD_DIR,
                           filename_suffix=f'OPT_{config.TRAIN_OPTIMIZER}_LR_{config.learning_rate}_BS_{config.batch}_Sub_{config.subdivisions}_Size_{config.width}',
                           comment=f'OPT_{config.TRAIN_OPTIMIZER}_LR_{config.learning_rate}_BS_{config.batch}_Sub_{config.subdivisions}_Size_{config.width}')
#     writer.add_images('legend',
#                       torch.from_numpy(train_dataset.label2colorlegend2(cfg.DATA_CLASSES).transpose([2, 0, 1])).to(
#                           device).unsqueeze(0))
    max_itr = config.TRAIN_EPOCHS * n_train
    # global_step = cfg.TRAIN_MINEPOCH * n_train
    global_step = 0
    logging.info(f'''Starting training:
        Epochs:          {epochs}
        Batch size:      {config.batch}
        Subdivisions:    {config.subdivisions}
        Learning rate:   {config.learning_rate}
        Training size:   {n_train}
        Validation size: {n_val}
        Checkpoints:     {save_cp}
        Device:          {device.type}
        Images size:     {config.width}
        Optimizer:       {config.TRAIN_OPTIMIZER}
        Dataset classes: {config.classes}
        Train label path:{config.train_label}
        Pretrained:
    ''')

    # learning rate setup
    def burnin_schedule(i):
        if i < config.burn_in:
            factor = pow(i / config.burn_in, 4)
        elif i < config.steps[0]:
            factor = 1.0
        elif i < config.steps[1]:
            factor = 0.1
        else:
            factor = 0.01
        return factor

    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate / config.batch, betas=(0.9, 0.999), eps=1e-08)
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, burnin_schedule)

    criterion = Custom_Yolo_loss(n_classes=config.classes, device=device, batch=config.batch // config.subdivisions)
    # scheduler = ReduceLROnPlateau(optimizer, mode='max', verbose=True, patience=6, min_lr=1e-7)
    # scheduler = CosineAnnealingWarmRestarts(optimizer, 0.001, 1e-6, 20)

    model.train()
    for epoch in range(epochs):
        #model.train()
        epoch_loss = 0
        epoch_step = 0

        with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img', ncols=50) as pbar:
            for i, batch in enumerate(train_loader):
                global_step += 1
                epoch_step += 1
                images = batch[0]
                bboxes = batch[1]

                images = images.to(device=device, dtype=torch.float32)
                bboxes = bboxes.to(device=device)

                bboxes_pred = model(images)
                loss, loss_xy, loss_wh, loss_obj, loss_cls, loss_l2 = criterion(bboxes_pred, bboxes)
                # loss = loss / config.subdivisions
                loss.backward()

                epoch_loss += loss.item()

                if global_step  % config.subdivisions == 0:
                    optimizer.step()
                    scheduler.step()
                    model.zero_grad()

                if global_step % (log_step * config.subdivisions) == 0:
                    writer.add_scalar('train/Loss', loss.item(), global_step)
                    writer.add_scalar('train/loss_xy', loss_xy.item(), global_step)
                    writer.add_scalar('train/loss_wh', loss_wh.item(), global_step)
                    writer.add_scalar('train/loss_obj', loss_obj.item(), global_step)
                    writer.add_scalar('train/loss_cls', loss_cls.item(), global_step)
                    writer.add_scalar('train/loss_l2', loss_l2.item(), global_step)
                    writer.add_scalar('lr', scheduler.get_lr()[0] * config.batch, global_step)
                    pbar.set_postfix(**{'loss (batch)': loss.item(), 'loss_xy': loss_xy.item(),
                                        'loss_wh': loss_wh.item(),
                                        'loss_obj': loss_obj.item(),
                                        'loss_cls': loss_cls.item(),
                                        'loss_l2': loss_l2.item(),
                                        'lr': scheduler.get_lr()[0] * config.batch
                                        })
                    logging.debug('Train step_{}: loss : {},loss xy : {},loss wh : {},'
                                  'loss obj : {}，loss cls : {},loss l2 : {},lr : {}'
                                  .format(global_step, loss.item(), loss_xy.item(),
                                          loss_wh.item(), loss_obj.item(),
                                          loss_cls.item(), loss_l2.item(),
                                          scheduler.get_lr()[0] * config.batch))

                pbar.update(images.shape[0])

            if save_cp:
                try:
                    os.mkdir(config.checkpoints)
                    logging.info('Created checkpoint directory')
                except OSError:
                    pass
                torch.save(model.state_dict(), os.path.join(config.checkpoints, f'Yolov4_epoch{epoch + 1}.pth'))
                logging.info(f'Checkpoint {epoch + 1} saved !')

    writer.close()
    

In [ ]:
class Custom_Yolo_loss(nn.Module):
    def __init__(self, n_classes=80, n_anchors=3, device=None, batch=2):
        super(Custom_Yolo_loss, self).__init__()
        self.device = device
        self.strides = [8, 16, 32]
        image_size = 416
        self.n_classes = n_classes
        self.n_anchors = n_anchors

        self.anchors = [[12, 16], [19, 36], [40, 28], [36, 75], [76, 55], [72, 146], [142, 110], [192, 243], [459, 401]]
        self.anch_masks = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
        self.ignore_thre = 0.5

        self.masked_anchors, self.ref_anchors, self.grid_x, self.grid_y, self.anchor_w, self.anchor_h = [], [], [], [], [], []

        for i in range(3):
            all_anchors_grid = [(w / self.strides[i], h / self.strides[i]) for w, h in self.anchors]
            masked_anchors = np.array([all_anchors_grid[j] for j in self.anch_masks[i]], dtype=np.float32)
            ref_anchors = np.zeros((len(all_anchors_grid), 4), dtype=np.float32)
            ref_anchors[:, 2:] = np.array(all_anchors_grid, dtype=np.float32)
            ref_anchors = torch.from_numpy(ref_anchors)
            # calculate pred - xywh obj cls
            fsize = image_size // self.strides[i]
            grid_x = torch.arange(fsize, dtype=torch.float).repeat(batch, 3, fsize, 1).to(device)
            grid_y = torch.arange(fsize, dtype=torch.float).repeat(batch, 3, fsize, 1).permute(0, 1, 3, 2).to(device)
            anchor_w = torch.from_numpy(masked_anchors[:, 0]).repeat(batch, fsize, fsize, 1).permute(0, 3, 1, 2).to(
                device)
            anchor_h = torch.from_numpy(masked_anchors[:, 1]).repeat(batch, fsize, fsize, 1).permute(0, 3, 1, 2).to(
                device)

            self.masked_anchors.append(masked_anchors)
            self.ref_anchors.append(ref_anchors)
            self.grid_x.append(grid_x)
            self.grid_y.append(grid_y)
            self.anchor_w.append(anchor_w)
            self.anchor_h.append(anchor_h)

    def build_target(self, pred, labels, batchsize, fsize, n_ch, output_id):

        # target assignment
        tgt_mask = torch.zeros(batchsize, self.n_anchors, fsize, fsize, 4 + self.n_classes).to(device=self.device)
        obj_mask = torch.ones(batchsize, self.n_anchors, fsize, fsize).to(device=self.device)
        tgt_scale = torch.zeros(batchsize, self.n_anchors, fsize, fsize, 2).to(self.device)
        target = torch.zeros(batchsize, self.n_anchors, fsize, fsize, n_ch).to(self.device)


            # labels = labels.cpu().data
        # nlabel = (labels.sum(dim=2) > 0).sum(dim=1)  # number of objects

        # batches_labels =  labels[:,-1]
        # nlabel = [(batches_labels == i).sum() for i in range(batchsize)]

        truth_x_all = (labels[:, 2] + labels[:, 0]) / (self.strides[output_id] * 2)
        truth_y_all = (labels[:, 3] + labels[:, 1]) / (self.strides[output_id] * 2)
        truth_w_all = (labels[:, 2] - labels[:, 0]) / self.strides[output_id]
        truth_h_all = (labels[:, 3] - labels[:, 1]) / self.strides[output_id]
        truth_i_all = truth_x_all.to(torch.int16).cpu().numpy()
        truth_j_all = truth_y_all.to(torch.int16).cpu().numpy()

        for b in range(batchsize):

            ## =====================================================================
            set_trace
            
            
            img_index = (labels[:,-1] == b).cpu().numpy()
            n = int(img_index.sum())
            if n == 0:
                continue

            truth_box = torch.zeros(n, 4).to(self.device)
            truth_box[:, 2] = truth_w_all[img_index]
            truth_box[:, 3] = truth_h_all[img_index]
            truth_i = truth_i_all[img_index]
            truth_j = truth_j_all[img_index]

            # calculate iou between truth and reference anchors
            anchor_ious_all = bboxes_iou(truth_box.cpu(), self.ref_anchors[output_id])
            best_n_all = anchor_ious_all.argmax(dim=1)
            best_n = best_n_all % 3
            best_n_mask = ((best_n_all == self.anch_masks[output_id][0]) |
                           (best_n_all == self.anch_masks[output_id][1]) |
                           (best_n_all == self.anch_masks[output_id][2]))

            if sum(best_n_mask) == 0:
                continue

            truth_box[:, 0] = truth_x_all[img_index]
            truth_box[:, 1] = truth_y_all[img_index]

            pred_ious = bboxes_iou(pred[b].view(-1, 4), truth_box, xyxy=False)
            pred_best_iou, _ = pred_ious.max(dim=1)
            pred_best_iou = (pred_best_iou > self.ignore_thre)
            pred_best_iou = pred_best_iou.view(pred[b].shape[:3])
            # set mask to zero (ignore) if pred matches truth
            obj_mask[b] = ~ pred_best_iou

            for ti in range(best_n.shape[0]):
                if best_n_mask[ti] == 1:
                    i, j = truth_i[ti], truth_j[ti]
                    a = best_n[ti]
                    obj_mask[b, a, j, i] = 1
                    tgt_mask[b, a, j, i, :] = 1
                    target[b, a, j, i, 0] = truth_x_all[ti] - truth_x_all[ti].to(torch.int16).to(torch.float)
                    target[b, a, j, i, 1] = truth_y_all[ti] - truth_y_all[ti].to(torch.int16).to(torch.float)
                    target[b, a, j, i, 2] = torch.log(
                        truth_w_all[ti] / torch.Tensor(self.masked_anchors[output_id])[best_n[ti], 0] + 1e-16)
                    target[b, a, j, i, 3] = torch.log(
                        truth_h_all[ti] / torch.Tensor(self.masked_anchors[output_id])[best_n[ti], 1] + 1e-16)
                    target[b, a, j, i, 4] = 1
                    target[b, a, j, i, 5 + labels[ti, 4].to(torch.int16).cpu().numpy()] = 1
                    tgt_scale[b, a, j, i, :] = torch.sqrt(2 - truth_w_all[ti] * truth_h_all[ti] / fsize / fsize)
        return obj_mask, tgt_mask, tgt_scale, target

    def forward(self, xin, labels=None):
        loss, loss_xy, loss_wh, loss_obj, loss_cls, loss_l2 = 0, 0, 0, 0, 0, 0
        for output_id, output in enumerate(xin):

            batchsize = output.shape[0]
            fsize = output.shape[2]

            n_ch = 5 + self.n_classes

            output = output.view(batchsize, self.n_anchors, n_ch, fsize, fsize)
            output = output.permute(0, 1, 3, 4, 2)  # .contiguous()

            # logistic activation for xy, obj, cls
            output[..., np.r_[:2, 4:n_ch]] = torch.sigmoid(output[..., np.r_[:2, 4:n_ch]])

            pred = output[..., :4].clone()
            pred[..., 0] += self.grid_x[output_id]
            pred[..., 1] += self.grid_y[output_id]
            pred[..., 2] = torch.exp(pred[..., 2]) * self.anchor_w[output_id]
            pred[..., 3] = torch.exp(pred[..., 3]) * self.anchor_h[output_id]

            ## get label for this batch only

            obj_mask, tgt_mask, tgt_scale, target = self.build_target(pred, labels, batchsize, fsize, n_ch, output_id)

            # loss calculation
            output[..., 4] *= obj_mask
            output[..., np.r_[0:4, 5:n_ch]] *= tgt_mask
            output[..., 2:4] *= tgt_scale

            target[..., 4] *= obj_mask
            target[..., np.r_[0:4, 5:n_ch]] *= tgt_mask
            target[..., 2:4] *= tgt_scale

            loss_xy += F.binary_cross_entropy(input=output[..., :2], target=target[..., :2],
                                              weight=tgt_scale * tgt_scale, size_average=False)
            loss_wh += F.mse_loss(input=output[..., 2:4], target=target[..., 2:4], size_average=False) / 2
            loss_obj += F.binary_cross_entropy(input=output[..., 4], target=target[..., 4], size_average=False)
            loss_cls += F.binary_cross_entropy(input=output[..., 5:], target=target[..., 5:], size_average=False)
            loss_l2 += F.mse_loss(input=output, target=target, size_average=False)

        loss = loss_xy + loss_wh + loss_obj + loss_cls

        return loss, loss_xy, loss_wh, loss_obj, loss_cls, loss_l2

train()

Epoch 1/300:   0%|       | 0/121 [00:00<?, ?img/s]C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\torch\cuda\nccl.py:24: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
Epoch 1/300:   2%| | 3/121 [00:01<01:03,  1.87img/C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\torch\optim\lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
Epoch 1/300:   2%| | 3/121 [00:02<01:03,  1.87img/--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\logging\__init__.py", line 1084, in emit
    stream.write(msg + sel

Epoch 1/300:   9%| | 11/121 [00:06<01:00,  1.82img--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\logging\__init__.py", line 1084, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\uff0c' in position 193: character maps to <undefined>
Call stack:
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\traitlets\config\ap

Epoch 1/300:  16%|▏| 19/121 [00:11<00:57,  1.77img--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\logging\__init__.py", line 1084, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\uff0c' in position 195: character maps to <undefined>
Call stack:
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\traitlets\config\ap

Epoch 1/300:  22%|▏| 27/121 [00:15<00:51,  1.83img--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\logging\__init__.py", line 1084, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\uff0c' in position 192: character maps to <undefined>
Call stack:
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\traitlets\config\ap

Epoch 1/300:  29%|▎| 35/121 [00:20<00:47,  1.81img--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\logging\__init__.py", line 1084, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\uff0c' in position 194: character maps to <undefined>
Call stack:
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\traitlets\config\ap

...

Epoch 7/300:  24%|▏| 29/121 [00:17<00:52,  1.76img--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\logging\__init__.py", line 1084, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\uff0c' in position 195: character maps to <undefined>
Call stack:
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\traitlets\config\ap

Epoch 7/300:  31%|▎| 37/121 [00:22<00:47,  1.77img--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\logging\__init__.py", line 1084, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\uff0c' in position 192: character maps to <undefined>
Call stack:
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\traitlets\config\ap

Epoch 7/300:  37%|▎| 45/121 [00:27<00:46,  1.62img--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\logging\__init__.py", line 1084, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\uff0c' in position 192: character maps to <undefined>
Call stack:
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\traitlets\config\ap

Epoch 7/300:  44%|▍| 53/121 [00:31<00:37,  1.80img--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\logging\__init__.py", line 1084, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\uff0c' in position 196: character maps to <undefined>
Call stack:
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Deployment\.conda\envs\yolo4\lib\site-packages\traitlets\config\ap

Epoch 7/300:  49%|▍| 59/121 [00:34<00:35,  1.76img

## Main

### Build DataLoader

In [19]:
from config.config import Cfg
data_list = "D:/YoloV5_Hui/Dataset/train_npt.txt"
dataset = Detection_dataset(data_list, Cfg)
dataset.names


## without augmentation
val_dataset = Detection_dataset(data_list, Cfg)

In [20]:
logging = init_logger(log_dir='log', stdout=False)

log file path:log\log_2020-07-07_14-00-51.txt


In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [22]:
cfg = get_args(**Cfg)
os.environ["CUDA_VISIBLE_DEVICES"] = cfg.gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')

model = Yolov4(cfg.pretrained,n_classes=cfg.classes)

if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)
model.to(device=device)

print()
cfg

{'batch': 4,
 'subdivisions': 4,
 'width': 416,
 'height': 416,
 'channels': 3,
 'momentum': 0.949,
 'decay': 0.0005,
 'angle': 0,
 'saturation': 1.1,
 'exposure': 1.1,
 'hue': 0.1,
 'learning_rate': 0.001,
 'burn_in': 1000,
 'max_batches': 500500,
 'steps': [400000, 450000],
 'policy': [400000, 450000],
 'scales': [0.1, 0.1],
 'cutmix': 0,
 'mosaic': 1,
 'letter_box': 0,
 'jitter': 0.2,
 'classes': 80,
 'track': 0,
 'w': 416,
 'h': 416,
 'flip': 1,
 'blur': 0,
 'gaussian': 0,
 'boxes': 60,
 'TRAIN_EPOCHS': 300,
 'train_label': 'train.txt',
 'val_label': 'data/val.txt',
 'TRAIN_OPTIMIZER': 'adam',
 'mixup': 3,
 'checkpoints': 'checkpoints',
 'TRAIN_TENSORBOARD_DIR': 'log',
 'load': 'C:\\Users\\Deployment\\AppData\\Roaming\\jupyter\\runtime\\kernel-6df6ad97-250d-4c03-8a64-b925044e8869.json',
 'gpu': '-1',
 'dataset_dir': None,
 'pretrained': None,
 'iou_type': 'iou',
 'keep_checkpoint_max': 10}

### Train

In [24]:
import random
import torch

def train():
    torch.manual_seed(20)
    np.random.seed(20)
    random.seed(20)


    try:
        custom_train(
            train_dataset=dataset,
            val_dataset=val_dataset,
            model=model,
            config=cfg,
            epochs=cfg.TRAIN_EPOCHS,
            device=device,
            log_step=1)
    except KeyboardInterrupt:
        torch.save(model.state_dict(), 'INTERRUPTED.pth')
        logging.info('Saved interrupt')
        try:
            sys.exit(0)
        except SystemExit:
            os._exit(0)
            
train()

Epoch 1/300:   0%|       | 0/121 [00:00<?, ?img/s]


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
import random
import torch
torch.manual_seed(20)
np.random.seed(20)
random.seed(20)

[len(dataset[i][1]) for i in range(5)]

## Export

In [ ]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "NOTEBOOK = " + "'"+thename+"'";
kernel.execute(command);

In [ ]:
!python notebook2script.py "$NOTEBOOK".ipynb

In [ ]:
%%javascripddt
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "NOTEBOOK = " + "'"+thename+"'";
kernel.execute(command);

!python notebook2script.py "$NOTEBOOK".ipynb